# Dev-Matching - 자연어 처리

데이터 로딩 및 토크나이징
    1. 트레이닝과 테스트 데이터 로딩
    2. Konlpy의 Okt를 사용해 데이터 토크나이징(보통 Punctuation을 제거하지만 컴퓨터
    언어에 Punctuation이 중요하기 때문에 제거하면 정확도가 많이 내려감.)
    3. 제목과 내용 둘다 사용해 토크나이징.(제목과 내용 둘다 사용해 corpus를 만들때 정확도가
    높아지는걸 볼수있었음.)

In [1]:
import pandas as pd
'''
Loading train and test data sets
'''
data = pd.read_csv('./data/hashcode_classification2020_train.csv')
test = pd.read_csv('./data/hashcode_classification2020_test.csv')
'''
Tokenize the train and test data with Okt from Konlpy
'''
from konlpy.tag import Okt

okt = Okt()
train_docs = []
test_docs = []

'''
Normally ruling out punctuation from the corpus is a good idea.
However, punctuation is important feature when it comes to programming languages.
(Accuracy score significantly goes down when punctuation is ruled out.)
'''
def tokenize(doc):
    result = []
    for t in okt.pos(doc, norm=True, stem=True):
        result.append('/'.join(t))
    return result
'''
Using both title and text to train the model.
Same with test set.
Also there is an error when tokenizing one of the data example,
used try and except to rule that one out. (If multiple data were giving errors, data cleaning would been necessary.)
'''
for idx, text in data.iterrows():
    try:
        train_docs.append((tokenize(text[0] + text[1]),text[2]))
    except:
        continue
for idx, text in test.iterrows():
    try:
        test_docs.append(tokenize(text[0]+text[1]))
    except:
        continue

print(train_docs[0])

(['wait/Alpha', '()/Punctuation', '과/Noun', 'sleep/Alpha', '()/Punctuation', '의/Noun', '차이점/Noun', '은/Josa', '뭔가/Noun', '요/Josa', '#/Foreign', '##발생하는/Hashtag', '문제/Noun', '및/Noun', '실행/Noun', '환경/Noun', '\n/Foreign', 'wait/Alpha', '()/Punctuation', '과/Noun', 'sleep/Alpha', '()/Punctuation', '의/Noun', '차이점/Noun', '은/Josa', '뭔가/Noun', '요/Josa'], 3)


# Doc2Vec이 요구하는 데이터 방식으로 바꿔주는 작업

In [2]:
'''
Doc2Vec requires the data to be formed in certain way.
Tagging the training document for our corpus model 
'''
from collections import namedtuple
TaggedDocument = namedtuple('TaggedDocument', 'words tags')
tagged_train_docs = [TaggedDocument(d, [c]) for d, c in train_docs]

print(tagged_train_docs[0])

TaggedDocument(words=['wait/Alpha', '()/Punctuation', '과/Noun', 'sleep/Alpha', '()/Punctuation', '의/Noun', '차이점/Noun', '은/Josa', '뭔가/Noun', '요/Josa', '#/Foreign', '##발생하는/Hashtag', '문제/Noun', '및/Noun', '실행/Noun', '환경/Noun', '\n/Foreign', 'wait/Alpha', '()/Punctuation', '과/Noun', 'sleep/Alpha', '()/Punctuation', '의/Noun', '차이점/Noun', '은/Josa', '뭔가/Noun', '요/Josa'], tags=[3])


# Doc2Vec으로 corpus를 트레이닝 하는 작업

doc2vec에 굉장히 많은 parameter들이 있어 많은 시도를 했음.

In [3]:
'''
dm = 0 means distributed bag of words(doesn't preserve the word order)
min_count = 3 means train model with words that appeared at least 3 times
(it is hard to learn from such a small set of data so min_count should be small)
hs = 0 using negative sample of 5 (negative = 5 by default)
verctor_size = 300 means 300 dimensionality of the feature vectors
sample = 0 is threshold for configuring which higher-frequency words are randomly downsampled.

'''
from gensim.models import doc2vec
import multiprocessing

cores = multiprocessing.cpu_count()
model = doc2vec.Doc2Vec(dm=0, vector_size=300, hs=0, negative = 5, min_count=3, sample = 0, workers=cores, seed=1234)

model.build_vocab(tagged_train_docs)

alpha = 0.025 # the initial learning rate
min_alpha = 0.001 # learning rate will linearly drop to min_alpha as training progresses
num_epochs = 20 # number of iterations over the corpus
alpha_delta = (alpha - min_alpha) / num_epochs
# train the model
for epoch in range(num_epochs):
    print('Now training epoch: ',epoch)
    model.alpha = alpha
    # fix the learning rate, no decay
    model.min_alpha = alpha
    model.train(tagged_train_docs, total_examples=model.corpus_count, epochs=model.epochs)
    # decrease the learning rate
    alpha -= alpha_delta

# Split the data into x,y train and x,y test
x_train = [model.infer_vector(doc.words) for doc in tagged_train_docs]
y_train = [doc.tags[0] for doc in tagged_train_docs]

x_test = [model.infer_vector(doc) for doc in test_docs]

Now training epoch:  0
Now training epoch:  1
Now training epoch:  2
Now training epoch:  3
Now training epoch:  4
Now training epoch:  5
Now training epoch:  6
Now training epoch:  7
Now training epoch:  8
Now training epoch:  9
Now training epoch:  10
Now training epoch:  11
Now training epoch:  12
Now training epoch:  13
Now training epoch:  14
Now training epoch:  15
Now training epoch:  16
Now training epoch:  17
Now training epoch:  18
Now training epoch:  19


# Multi Layer Perceptron Model (MLP) 만들어 학습후 예측결과 저장

In [4]:
import sklearn.metrics as metrics
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
import numpy as np

'''
The keras to_categorical expects the category value to start from 0.
Had to subtract one since our model starts from 1 - 5.
'''
#레이블 데이터를 one-hot형식으로 변경해줘야함
for i in range(len(y_train)):
    y_train[i] -= 1
y_train = keras.utils.np_utils.to_categorical(y_train, 5)

#입력과 출력 벡터의 크기를 정확히 지정해야함
in_size = x_train[0].shape[0]
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
'''
Added dropout value 0.2 to prevent overfitting.
It drops out both hidden and visible value from the model by probability of 1 - p or p.
'''
#MLP모델 구조 정의
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(in_size,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))

'''
Used RMSprop, Adam, SGD but got best result using RMSprop
RMSprop is good, fast and very popular optimizer.
'''
#모델 컴파일하기
model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(),
    metrics=['accuracy'])
#학습하기
hist = model.fit(x_train, y_train,
          batch_size=128, 
          epochs=100,
          verbose=1)
'''
make prediction from the model we trained and save as prediction.csv
before saving, add 1 back to the prediction since we subtracted one to fit into keras category
'''
predictions = model.predict_classes(x_test)

for i in range(len(predictions)):
    predictions[i] += 1
pd.DataFrame(predictions, columns=['label']).to_csv('prediction.csv', index=False)

Using TensorFlow backend.






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/100





2591/2591 [==============================] - 1s 423us/step - loss: 0.9703 - acc: 0.6515
Epoch 2/100
2591/2591 [==============================] - 0s 112us/step - loss: 0.5489 - acc: 0.8109
Epoch 3/100
2591/2591 [==============================] - 0s 113us/step - loss: 0.4331 - acc: 0.8580
Epoch 4/100
2591/2591 [==============================] - 0s 113us/step - loss: 0.3791 - acc: 0.8703
Epoch 5/100
2591/2591 [==============================] - 0s 114us/step - loss: 0.3667 - acc: 0.8738
Epoch 6/100
2591/2591 [==============================] - 0s 114us/step - loss: 0.3242 - acc: 0.8885
Epoch 7/100
2591/2591 [==============================] - 0s 123us/step - loss: 0.3192 - acc: 0.8823
Epoch 8/100
2591/2591 [==============================] - 0s 116us/step - loss

2591/2591 [==============================] - 0s 126us/step - loss: 0.1601 - acc: 0.9448
Epoch 48/100
2591/2591 [==============================] - 0s 126us/step - loss: 0.1402 - acc: 0.9475
Epoch 49/100
2591/2591 [==============================] - 0s 127us/step - loss: 0.1512 - acc: 0.9425
Epoch 50/100
2591/2591 [==============================] - 0s 127us/step - loss: 0.1483 - acc: 0.9456
Epoch 51/100
2591/2591 [==============================] - 0s 126us/step - loss: 0.1367 - acc: 0.9483
Epoch 52/100
2591/2591 [==============================] - 0s 127us/step - loss: 0.1242 - acc: 0.9560
Epoch 53/100
2591/2591 [==============================] - 0s 126us/step - loss: 0.1406 - acc: 0.9506
Epoch 54/100
2591/2591 [==============================] - 0s 129us/step - loss: 0.1347 - acc: 0.9498
Epoch 55/100
2591/2591 [==============================] - 0s 128us/step - loss: 0.1327 - acc: 0.9506
Epoch 56/100
2591/2591 [==============================] - 0s 133us/step - loss: 0.1513 - acc: 0.9398
Epo